# <b>1,2회차 실습 예제</b>

* 총 6단계로 1,2회차 실습으로 나눠서 진행
    1. 파이썬 실치 해보기 (skip 가능) or colab등으로 실습 환경 마련가능
    2. 데이터셋을 만들어야합니다.
    3. 데이터셋 시각화
    4. 다음 실습때... 학습 제대로
    5. 
    6. 
* 만들어 볼 것은 이미지 분류기(classifier)입니다.
    * 데이터 수집/정제
    * 인공지능 모델 학습
    * 학습된 모델 배포

# 1회차 실습 코드

환경 구성 3가지 케이스


1.코랩을 사용하면 설치하지 않아도 됩니다.  
2. vscode 설치시 필요한 플러그인 자동 설치 됩니다.

[설치 설명](https://arusantimo.notion.site/b0992d1aee0d42ae9f2cbd8ac730a481)

## 설치 과정

In [ ]:
# 한글 폰트 설치하기 (한글 유니코드 깨짐 방지)
!pip install koreanize-matplotlib

In [ ]:
import matplotlib.pyplot as plt
import koreanize_matplotlib

In [ ]:
# 필요한 라이브러리 설치하기 이미지 수집을 위한
!git clone https://github.com/ndb796/bing_image_downloader

#### <b>1. 학습용 데이터셋 만들기 - 이미지 크롤링을 활용한 학습 이미지 수집</b>

* 이미지를 일일히 다운받는것은 너무 비효율적이기 때문에 간단한 크롤러를 활욜해서 데이터를 수집합니다. 추후 생성형 AI를 통해 데이터셋을 무수히 재 생산할수 있습니다.

In [ ]:
import os
import shutil
from bing_image_downloader.bing_image_downloader import downloader

# 이미지 저장 디렉토리 지정
directory_list = [
    './custom_dataset/train/',
    './custom_dataset/test/',
]

# 초기 디렉토리 만들기
for directory in directory_list:
    if not os.path.isdir(directory):
        os.makedirs(directory)

# 수집한 이미지를 학습 데이터와 평가 데이터로 구분하는 함수
def dataset_split(query, train_cnt):
    # 학습 및 평가 데이터셋 디렉토리 만들기
    for directory in directory_list:
        if not os.path.isdir(directory + '/' + query):
            os.makedirs(directory + '/' + query)
    # 학습 및 평가 데이터셋 준비하기
    cnt = 0
    for file_name in os.listdir(query):
        if cnt < train_cnt:
            print(f'[Train Dataset] {file_name}')
            shutil.move(query + '/' + file_name, './custom_dataset/train/' + query + '/' + file_name)
        else:
            print(f'[Test Dataset] {file_name}')
            shutil.move(query + '/' + file_name, './custom_dataset/test/' + query + '/' + file_name)
        cnt += 1
    shutil.rmtree(query)

* <b>검색대상 - [송혜교] 아무나 가능</b> 이미지 크롤링을 진행하고 데이터셋을 구축합니다.

In [ ]:
query = '송혜교'
downloader.download(query, limit=40,  output_dir='./', adult_filter_off=True, force_replace=False, timeout=60)
dataset_split(query, 30)

* <b>다른 캐릭터</b> 이미지 크롤링을 진행하고 데이터셋을 구축합니다.

In [ ]:
query = '송태섭'
downloader.download(query, limit=40,  output_dir='./', adult_filter_off=True, force_replace=False, timeout=60)
dataset_split(query, 30)

* <b>마지막 캐릭터</b> 이미지 크롤링을 진행하고 데이터셋을 구축합니다.

In [ ]:
query = '도구리'
downloader.download(query, limit=40,  output_dir='./', adult_filter_off=True, force_replace=False, timeout=60)
dataset_split(query, 30)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

import torchvision
from torchvision import datasets, models, transforms

import numpy as np
import time

# 학습을 위한 데이터 셋 정리용 라이브러리 들
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") # device 객체

In [ ]:
transforms_train = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(), # 데이터 증진(augmentation)
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) # 정규화(normalization)
])

transforms_test = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

data_dir = './custom_dataset'
train_datasets = datasets.ImageFolder(os.path.join(data_dir, 'train'), transforms_train)
test_datasets = datasets.ImageFolder(os.path.join(data_dir, 'test'), transforms_test)

train_dataloader = torch.utils.data.DataLoader(train_datasets, batch_size=4, shuffle=True, num_workers=4)
test_dataloader = torch.utils.data.DataLoader(test_datasets, batch_size=4, shuffle=True, num_workers=4)

print('학습 데이터셋 크기:', len(train_datasets))
print('테스트 데이터셋 크기:', len(test_datasets))

class_names = train_datasets.classes
print('클래스:', class_names)

In [ ]:
def imshow(input, title):
    # torch.Tensor를 numpy 객체로 변환
    input = input.numpy().transpose((1, 2, 0))
    # 이미지 정규화 해제하기
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    input = std * input + mean
    input = np.clip(input, 0, 1)
    # 이미지 출력
    plt.imshow(input)
    plt.title(title)
    plt.show()


# 학습 데이터를 배치 단위로 불러오기
iterator = iter(train_dataloader)

# 현재 배치를 이용해 격자 형태의 이미지를 만들어 시각화
inputs, classes = next(iterator)
out = torchvision.utils.make_grid(inputs)
imshow(out, title=[class_names[x] for x in classes])